In [16]:
import os
import datetime
import subprocess
import csv
import smtplib
import zipfile
from email.mime.text import MIMEText
import glob

# Configurations
DB_NAME = "facility_db"
DB_USER = "root"
DB_PASS = "root"

FACILITY_NAME = "FacilityA"
BASE_BACKUP_DIR = "C:\\Users\\chipu\\Documents\\ZIMTTECH\\Facility Backup"
LOG_FILE = "C:\\Users\\chipu\\Documents\\ZIMTTECH\\Facility Backup\\Back up logs\\backup_log.csv"

EMAIL_FROM = "facility.backup.system@gmail.com"
EMAIL_TO = "chipurirowalterc@gmail.com"
EMAIL_PASS = "facility123"

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

ZIP_PASSWORD = b"S3cur3B@ckup2025!"

# ===== FUNCTIONS =====

def send_email_alert(subject, msg):
    message = MIMEText(msg)
    message['Subject'] = subject
    message['From'] = EMAIL_FROM
    message['To'] = EMAIL_TO
    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_FROM, EMAIL_PASS)
            server.send_message(message)
        print("✅ Email alert sent.")
    except Exception as e:
        print("❌ Failed to send alert email:", e)

def zip_and_encrypt(filepath):
    zip_path = filepath + ".zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.setpassword(ZIP_PASSWORD)
        zf.write(filepath, arcname=os.path.basename(filepath))
    os.remove(filepath)
    print(f"🔐 Encrypted zip created: {zip_path}")
    return zip_path

def log_backup(timestamp, btype, status, filename, filesize_mb, duration_sec, message):
    file_exists = os.path.exists(LOG_FILE)
    with open(LOG_FILE, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["timestamp", "type", "status", "filename", "file_size_mb", "duration_sec", "message"])
        writer.writerow([timestamp, btype, status, filename, round(filesize_mb, 2), duration_sec, message])


MYSQLDUMP_PATH = r"C:\Program Files\MySQL\MySQL Server 9.1\bin\mysqldump.exe"

def run_mysqldump(output_path, incremental_since=None):
    print(f"🟡 Running {'incremental' if incremental_since else 'full'} backup to {output_path}")

    if incremental_since:
        # Customize incremental logic here, demo uses full dump
        cmd = [MYSQLDUMP_PATH, "-u", DB_USER, f"-p{DB_PASS}", DB_NAME]
    else:
        cmd = [MYSQLDUMP_PATH, "-u", DB_USER, f"-p{DB_PASS}", DB_NAME]

    start_time = datetime.datetime.now()
    with open(output_path, "w") as f:
        result = subprocess.run(cmd, stdout=f, stderr=subprocess.PIPE)
    end_time = datetime.datetime.now()

    status = "success" if result.returncode == 0 else "failed"
    filesize_mb = os.path.getsize(output_path) / (1024 * 1024) if os.path.exists(output_path) else 0
    duration = (end_time - start_time).seconds
    message = "Backup completed successfully." if status == "success" else result.stderr.decode().strip()

    return status, filesize_mb, duration, message


# def run_mysqldump(output_path, incremental_since=None):
#     print(f"🟡 Running {'incremental' if incremental_since else 'full'} backup to {output_path}")

#     if incremental_since:
#         # Customize your incremental logic here, for demo dumping full DB
#         cmd = ["mysqldump", "-u", DB_USER, f"-p{DB_PASS}", DB_NAME]
#     else:
#         cmd = ["mysqldump", "-u", DB_USER, f"-p{DB_PASS}", DB_NAME]

#     start_time = datetime.datetime.now()
#     with open(output_path, "w") as f:
#         result = subprocess.run(cmd, stdout=f, stderr=subprocess.PIPE)
#     end_time = datetime.datetime.now()

#     status = "success" if result.returncode == 0 else "failed"
#     filesize_mb = os.path.getsize(output_path) / (1024 * 1024) if os.path.exists(output_path) else 0
#     duration = (end_time - start_time).seconds
#     message = "Backup completed successfully." if status == "success" else result.stderr.decode().strip()

#     return status, filesize_mb, duration, message

def get_last_full_backup():
    full_files = sorted(glob.glob(os.path.join(BASE_BACKUP_DIR, f"{FACILITY_NAME}_full_*.sql.zip")))
    return full_files[-1] if full_files else None

def get_incremental_backups_for(full_backup_zip):
    if not full_backup_zip:
        return []
    full_time_str = os.path.basename(full_backup_zip).split("_full_")[1].replace(".sql.zip", "")
    full_time = datetime.datetime.strptime(full_time_str, "%Y-%m-%d_%H-%M-%S")

    incr_files = glob.glob(os.path.join(BASE_BACKUP_DIR, f"{FACILITY_NAME}_incr_*.sql.zip"))
    filtered = []
    for fpath in incr_files:
        time_str = os.path.basename(fpath).split("_incr_")[1].replace(".sql.zip", "")
        try:
            ts = datetime.datetime.strptime(time_str, "%Y-%m-%d_%H-%M-%S")
            if ts > full_time:
                filtered.append(fpath)
        except:
            continue
    return sorted(filtered)

def delete_old_backups(full_backup_to_keep):
    full_backups = glob.glob(os.path.join(BASE_BACKUP_DIR, f"{FACILITY_NAME}_full_*.sql.zip"))
    full_backups = sorted(full_backups, reverse=True)
    for fb in full_backups:
        if fb != full_backup_to_keep:
            print(f"🗑️ Deleting old full backup: {fb}")
            os.remove(fb)
            incrs = get_incremental_backups_for(fb)
            for incr in incrs:
                print(f"🗑️ Deleting incremental backup: {incr}")
                os.remove(incr)

def do_full_backup():
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    btype = "full"
    filename = f"{FACILITY_NAME}_full_{timestamp}.sql"
    filepath = os.path.join(BASE_BACKUP_DIR, filename)

    status, filesize_mb, duration, message = run_mysqldump(filepath, incremental_since=None)

    if status == "success":
        zip_path = zip_and_encrypt(filepath)
        log_backup(timestamp, btype, status, os.path.basename(zip_path), filesize_mb, duration, message)
        print(f"✅ Full backup done and zipped: {zip_path}")
        delete_old_backups(zip_path)
    else:
        send_email_alert("🚨 Facility Backup FAILED", f"Full backup failed at {timestamp}.\n\nError:\n{message}")

def main():
    today = datetime.datetime.now()
    weekday = today.weekday()  # Monday=0, Sunday=6

    timestamp = today.strftime("%Y-%m-%d_%H-%M-%S")

    if weekday == 6:  # Sunday full backup
        do_full_backup()
    else:
        btype = "incr"
        last_full = get_last_full_backup()
        if not last_full:
            print("⚠️ No full backup found! Performing full backup instead.")
            do_full_backup()
            return

        filename = f"{FACILITY_NAME}_incr_{timestamp}.sql"
        filepath = os.path.join(BASE_BACKUP_DIR, filename)

        incremental_since = None  # implement as needed

        status, filesize_mb, duration, message = run_mysqldump(filepath, incremental_since=incremental_since)
        if status == "success":
            zip_path = zip_and_encrypt(filepath)
            log_backup(timestamp, btype, status, os.path.basename(zip_path), filesize_mb, duration, message)
            print(f"✅ Incremental backup done and zipped: {zip_path}")
        else:
            send_email_alert("🚨 Facility Backup FAILED", f"Incremental backup failed at {timestamp}.\n\nError:\n{message}")

if __name__ == "__main__":
    main()


⚠️ No full backup found! Performing full backup instead.
🟡 Running full backup to C:\Users\chipu\Documents\ZIMTTECH\Facility Backup\FacilityA_full_2025-07-01_14-58-49.sql
🔐 Encrypted zip created: C:\Users\chipu\Documents\ZIMTTECH\Facility Backup\FacilityA_full_2025-07-01_14-58-49.sql.zip
✅ Full backup done and zipped: C:\Users\chipu\Documents\ZIMTTECH\Facility Backup\FacilityA_full_2025-07-01_14-58-49.sql.zip
